<a href="https://colab.research.google.com/github/NongNNew/Project_499/blob/main/Project_499_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CNN

In [1]:
!git clone https://github.com/NongNNew/Project_499.git

Cloning into 'Project_499'...
remote: Enumerating objects: 683, done.
remote: Counting objects: 100% (235/235), done.
remote: Compressing objects: 100% (232/232), done.
remote: Total 683 (delta 15), reused 215 (delta 3), pack-reused 448
Receiving objects: 100% (683/683), 118.10 MiB | 8.26 MiB/s, done.
Resolving deltas: 100% (43/43), done.
Checking out files: 100% (562/562), done.


In [2]:
import pandas as pd
from glob import glob
import librosa
import librosa.display

In [19]:
fruit_veget = ['กระชาย','กระท้อน','กระเทียม1','กระเทียม2','กระเพรา','กล้วยน้ำว้า','กล้วยหอม',
               'ข้าวโพด','ไข่น้ำ',
               'ตะขบไทย','ตะไคร้',
               'ถั่วฝักยาว','ถั่วลันเตา','ถั่วลิสง',
               'ทับทิม',
               'น้อยหน่า','น้ำเต้า',
               'บวบ','ใบชะพลู1','ใบชะพลู2','ใบบัวบก','ใบแมงลัก',
               'ฝรั่ง',
               'พุทรา',
               'มะปราง','มะพลับ','มะละกอ',
               'สับปะรด']

source = {'audio_time':[],
          'sampling_rate':[],
          'feature_extraction':[],
          'label':[],
          'description':[]}

In [20]:
for types in fruit_veget:
    audio_files = glob('/content/Project_499/Record_audio/' + str(types) + '/*.wav')

    for audio in audio_files:
        y,sr = librosa.load(audio,duration=6,offset=0)
        source['description'].append(str(types))
        source['audio_time'].append(y)
        source['sampling_rate'].append(sr)
        source['feature_extraction'].append(librosa.feature.mfcc(y,sr,n_mfcc=13))
        source['label'].append(fruit_veget.index(types))

In [21]:
import re

for i in range(len(source['description'])):
    source['description'][i] = re.findall('[\u0E00-\u0E7F]+',source['description'][i])[0]

In [22]:
pd.DataFrame.from_dict(source)

,audio_time,sampling_rate,feature_extraction,label,description
0,"[-0.0001373291, -0.00010681152, -0.0001373291,...",22050,"[[-641.21436, -640.2155, -639.54114, -638.4883...",0,กระชาย
1,"[0.0004119873, 0.00061035156, 0.0005340576, 0....",22050,"[[-625.25085, -626.8685, -629.381, -632.6441, ...",0,กระชาย
2,"[-9.1552734e-05, -0.00012207031, -0.0001373291...",22050,"[[-644.627, -645.0809, -645.80524, -646.2903, ...",0,กระชาย
3,"[-0.00010681152, -0.00010681152, -0.0001525878...",22050,"[[-622.78125, -623.396, -623.63983, -623.7701,...",0,กระชาย
4,"[-0.0002593994, -0.00022888184, -0.0002593994,...",22050,"[[-639.25616, -639.47345, -640.0064, -640.5056...",0,กระชาย
...,...,...,...,...,...
555,"[-0.00018310547, -0.00024414062, -0.0002288818...",22050,"[[-622.7785, -621.6811, -618.9255, -619.49744,...",27,สับปะรด
556,"[-0.00018310547, -0.0002593994, -0.00022888184...",22050,"[[-666.57153, -663.4788, -662.98236, -663.7374...",27,สับปะรด
557,"[0.0, 0.0, 0.0, 3.0517578e-05, -4.5776367e-05,...",22050,"[[-644.81506, -644.89075, -644.7345, -643.6741...",27,สับปะรด
558,"[-0.00010681152, -0.00019836426, -0.0002441406...",22050,"[[-688.85156, -688.6841, -685.51434, -680.2233...",27,สับปะรด


In [28]:
import tensorflow as tf
from tensorflow import keras 
import numpy as np
from sklearn.model_selection import train_test_split

X = np.array(source['feature_extraction'])
y = np.array(source['label'])

X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.25)

model = keras.Sequential()
model.add(keras.layers.Conv2D(64, 5, input_shape=(13, 259, 1), activation='relu'))
model.add(keras.layers.Dropout(0.25))
model.add(keras.layers.MaxPool2D())
model.add(keras.layers.Conv2D(64, 5, activation='relu',padding='same'))
model.add(keras.layers.Dropout(0.25))
model.add(keras.layers.MaxPool2D())
model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(128, activation='relu'))
model.add(keras.layers.Dropout(0.5))
model.add(keras.layers.Dense(len(np.unique(source['label'])), activation='softmax'))
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [29]:
model.fit(X_train[:,:,:,None], y_train, epochs=100)

Epoch 1/100
14/14 [==============================] - 1s 23ms/step - loss: 44.6739 - accuracy: 0.0333
Epoch 2/100
14/14 [==============================] - 0s 20ms/step - loss: 3.3184 - accuracy: 0.0500
Epoch 3/100
14/14 [==============================] - 0s 19ms/step - loss: 3.3114 - accuracy: 0.0571
Epoch 4/100
14/14 [==============================] - 0s 18ms/step - loss: 3.3496 - accuracy: 0.0524
Epoch 5/100
14/14 [==============================] - 0s 18ms/step - loss: 3.3301 - accuracy: 0.0476
Epoch 6/100
14/14 [==============================] - 0s 18ms/step - loss: 3.3215 - accuracy: 0.0405
Epoch 7/100
14/14 [==============================] - 0s 18ms/step - loss: 3.3015 - accuracy: 0.0452
Epoch 8/100
14/14 [==============================] - 0s 19ms/step - loss: 3.3158 - accuracy: 0.0548
Epoch 9/100
14/14 [==============================] - 0s 18ms/step - loss: 3.2917 - accuracy: 0.0452
Epoch 10/100
14/14 [==============================] - 0s 19ms/step - loss: 3.2970 - accuracy: 0.052

In [30]:
model.summary()

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_12 (Conv2D)           (None, 9, 255, 64)        1664      
_________________________________________________________________
dropout_18 (Dropout)         (None, 9, 255, 64)        0         
_________________________________________________________________
max_pooling2d_12 (MaxPooling (None, 4, 127, 64)        0         
_________________________________________________________________
conv2d_13 (Conv2D)           (None, 4, 127, 64)        102464    
_________________________________________________________________
dropout_19 (Dropout)         (None, 4, 127, 64)        0         
_________________________________________________________________
max_pooling2d_13 (MaxPooling (None, 2, 63, 64)         0         
_________________________________________________________________
flatten_6 (Flatten)          (None, 8064)             

In [31]:
model.evaluate(X_test[:,:,:,None],y_test)

5/5 [==============================] - 0s 9ms/step - loss: 0.4880 - accuracy: 0.9286


[0.48796772956848145, 0.9285714030265808]